# 퍼셉트론 (Perceptron)
> 인공신경망의 한 종류로서, 출력이 0 또는 1인 작업을 의미하는 이진 분류 작업에 사용되는 간단한 모델이다. 페셉트론은 신경 세포(Neuron)가 신호를 전달하는 구조와 유사한 방식으로 구현됐다.

## 단층 퍼셉트론(Single Layer Perceptron)
> 입력을 통해 데이터가 전달되고, 입력값은 각각의 가중치와 함께 노드에 전달된다. 전달된 입력값과 가중치를 곱합 값이 활성화 함수에 전달된다. 활성화 함수에서 출력값이 계산되고 이 값을 손실함수에 실제값과 함께 연산해 가중치를 변경한다.

- `단층 퍼셉트론의 한계`: AND, OR, NAND 게이트와 같은 구조를 갖는 모델은 쉽게 구현하지만 XOR 게이트처럼 하나의 기울기로 표현하기 어려운 구조에서는 적응이 어렵다.


In [1]:
# 데이터셋에서 x1, x2는 입력값, y는 XOR 게이트를 통과했을 때의 결과를 의미

import torch
import pandas as pd
from torch import nn
from torch import optim
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
  def __init__(self, file_path):
    df = pd.read_csv(file_path)
    self.x1 = df.iloc[:, 0].values
    self.x2 = df.iloc[:, 1].values
    self.y = df.iloc[:, 2].values
    self.length = len(df)
    
  def __getitem__(self, index):
    x = torch.FloatTensor([self.x1[index], self.x2[index]])
    y = torch.FloatTensor([self.y[index]])
    return x, y
  
  def __len__(self):
    return self.length
    
class CustomModel(nn.Module):
  def __init__(self):
    super().__init__()
    
    self.layer = nn.Sequential(
      nn.Linear(2,1),
      nn.Sigmoid()
    )
  def forward(self, x):
    x = self.layer(x)
    return x
  
train_dataset = CustomDataset('../dataset/perceptron.csv')
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True, drop_last=True)

device = 'mps'
model = CustomModel().to(device)
criterion = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

for epoch in range(10000):
  cost = 0.0
  
  for x, y in train_dataloader:
    x = x.to(device)
    y = y.to(device)
    
    output = model(x)
    loss = criterion(output, y)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    cost += loss
  cost = cost / len(train_dataloader)
  
  if (epoch + 1) % 1000 == 0:
    print(f'Epoch: {epoch + 1:04d}, Cost: {cost:.6f}')
    
with torch.no_grad():
  model.eval()
  inputs = torch.FloatTensor([[0, 0], [0, 1], [1, 0], [1, 1]]).to(device)
  outputs = model(inputs)
  
  print("--------------------------------")
  print(outputs)
  print(outputs <= 0.5)


/var/folders/2y/2cp_6ft96fn709961nxvzc8h0000gn/T/ipykernel_36169/2278278290.py:18: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  x = torch.FloatTensor([self.x1[index], self.x2[index]])
/var/folders/2y/2cp_6ft96fn709961nxvzc8h0000gn/T/ipykernel_36169/2278278290.py:19: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  y = torch.FloatTensor([self.y[index]])


Epoch: 1000, Cost: 0.692743
Epoch: 2000, Cost: 0.691857
Epoch: 3000, Cost: 0.692155
Epoch: 4000, Cost: 0.692033
Epoch: 5000, Cost: 0.691577
Epoch: 6000, Cost: 0.692609
Epoch: 7000, Cost: 0.691862
Epoch: 8000, Cost: 0.691921
Epoch: 9000, Cost: 0.691986
Epoch: 10000, Cost: 0.691038
--------------------------------
tensor([[0.4672],
        [0.4998],
        [0.5030],
        [0.5356]], device='mps:0')
tensor([[ True],
        [ True],
        [False],
        [False]], device='mps:0')


# 다층 퍼셉트론(Multi_Layer Perceptron, MLP)
> 단층 퍼셉트론을 여러 개 쌓아 은닉층을 생성한다. 은닉층을 2개 이상 연결하면 심층 신경망 이라 부른다. 계층이 많으면 더 정확한 값을 찾을 수 있다. 하지만 계층이 늘어날수록 갱신해야 하는 가중치나 편향이 늘어난다. 최적의 가중치와 편향을 찾기 위해 많은 학습 데이터와 연산량을 필요로 한다.

In [3]:
class CustomModel(nn.Module):
  def __init__(self):
    super().__init__()
    
    self.layer1 = nn.Sequential(
      nn.Linear(2, 2),
      nn.Sigmoid()
    )
    
    self.layer2 = nn.Sequential(
      nn.Linear(2, 1),
      nn.Sigmoid()
    )
  def forward(self, x):
    x = self.layer1(x)
    x = self.layer2(x)
    return x
  
model = CustomModel().to(device)
criterion = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

for epoch in range(10000):
  cost = 0.0
  
  for x, y in train_dataloader:
    x = x.to(device)
    y = y.to(device)
    
    output = model(x)
    loss = criterion(output, y)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    cost += loss
  cost = cost / len(train_dataloader)
  
  if (epoch + 1) % 1000 == 0:
    print(f'Epoch: {epoch + 1:04d}, Cost: {cost:.6f}')
    
with torch.no_grad():
  model.eval()
  inputs = torch.FloatTensor([[0, 0], [0, 1], [1, 0], [1, 1]]).to(device)
  outputs = model(inputs)
  
  print("--------------------------------")
  print(outputs)
  print(outputs <= 0.5)

/var/folders/2y/2cp_6ft96fn709961nxvzc8h0000gn/T/ipykernel_36169/2278278290.py:18: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  x = torch.FloatTensor([self.x1[index], self.x2[index]])
/var/folders/2y/2cp_6ft96fn709961nxvzc8h0000gn/T/ipykernel_36169/2278278290.py:19: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  y = torch.FloatTensor([self.y[index]])


Epoch: 1000, Cost: 0.693472
Epoch: 2000, Cost: 0.693049
Epoch: 3000, Cost: 0.689960
Epoch: 4000, Cost: 0.622948
Epoch: 5000, Cost: 0.488062
Epoch: 6000, Cost: 0.099369
Epoch: 7000, Cost: 0.041701
Epoch: 8000, Cost: 0.025805
Epoch: 9000, Cost: 0.018531
Epoch: 10000, Cost: 0.014431
--------------------------------
tensor([[0.0164],
        [0.9865],
        [0.9876],
        [0.0150]], device='mps:0')
tensor([[ True],
        [False],
        [False],
        [ True]], device='mps:0')
